In [ ]:
# imports and constants
import os
import glob
import nibabel as nib
import nilearn
import numpy as np
import tensorflow as tf
import random
import scipy
from tensorflow import keras
# from keras import Sequential
# from keras.layers import Dense, Flatten, Dropout, Reshape, Conv3D, Conv2D, Activation, UpSampling3D, \
#                          MaxPooling3D, SpatialDropout3D, BatchNormalization, Conv3DTranspose
# from keras.layers.merge import concatenate
# from keras.engine import Input, Model
# from keras.optimizers import Adam
# from keras import backend as K
from matplotlib import pyplot as plt
# from sklearn.model_selection import train_test_split
from nilearn import image
from datetime import datetime
from scipy import ndimage
# import keras_unet
import imageio


DEBUG = False # for testing purposes
DEBUG_SAMPLES = 80

print(tf.test.is_gpu_available())

TRAINING_DATA_ROOT_DIR = "data/"
TRAIN_DIRS = ["train/LGG", "train/HGG"]
VAL_DIRS = ["val/val"]

SEQUENCES = ["t1ce"]
GROUND_TRUTH_FILE = "seg"

TYPE_OF_SEGMENTATION = "ET"

N_X = 240
N_Y = 240
N_Z = 155
NUM_CHANNELS = len(SEQUENCES)
NUM_CLASSES = 2


PATCH_X = 80
PATCH_Y = 80
PATCH_Z = 80


BATCH_SIZE = 2

PERCENTILE_95 = 10.204082 * 2 # 20 pixels

In [ ]:
nii_img_tc_seg  = nib.load('Brats18_TCIA03_288_1/Brats18_TCIA03_288_1_tc_t1ce_seg.nii.gz').get_fdata()

In [ ]:
nii_img_et_seg  = nib.load('Brats18_TCIA03_288_1/Brats18_TCIA03_288_1_et_t1ce_seg.nii.gz').get_fdata()

for i in range(240):
    for j in range(240):
        for k in range(155):
            if nii_img_et_seg[i][j][k] == 4:
                nii_img_et_seg[i][j][k] = 1

In [ ]:
nii_img_orig_t1ce  = nib.load('Brats18_TCIA03_288_1/Brats18_TCIA03_288_1_t1ce.nii.gz').get_fdata()

In [ ]:
nii_img_orig_flair  = nib.load('Brats18_TCIA03_288_1/Brats18_TCIA03_288_1_flair.nii.gz').get_fdata()

In [ ]:
orig_img = nib.load('../image-segmentation/data/val/val/Brats18_TCIA03_288_1/Brats18_TCIA03_288_1_t2.nii.gz').get_fdata()

In [ ]:
plt.imshow(orig_img[:, :, 65])

In [ ]:
nii_img_tc_seg.shape

In [ ]:
nii_img_orig_flair.shape

In [ ]:
np.sum(nii_img_tc_seg[:, :, 65])

In [ ]:
plt.imshow(nii_img_et_seg[:, :, 65], cmap='gray')

In [ ]:
np.unique(nii_img_orig_t1ce[:, :, 65])

In [ ]:
combined_mask = np.zeros((240, 240, 4))

for i in range(240):
    for j in range(240):
        combined_mask[i][j] = (0, 0, 0, 0)
        if nii_img_tc_seg[i][j][65] == 1 and nii_img_et_seg[i][j][65] == 1:
            combined_mask[i][j] = (0.9,0.1,0.1, 0.9)
            continue
        if nii_img_tc_seg[i][j][65] == 1 and nii_img_et_seg[i][j][65] == 0:
            combined_mask[i][j] = (0.999,0.999, 0.999, 0.999)
            continue
#         if nii_img_tc_seg[i][j][65] == 0 and nii_img_et_seg[i][j][65] == 1:
#             combined_mask[i][j] = (0, 255, 0, 0.75)
#             continue

In [ ]:
np.unique(combined_mask)

In [ ]:
nii_img_orig_t1ce  = nib.load('Brats18_TCIA03_288_1/Brats18_TCIA03_288_1_t1ce.nii.gz').get_fdata()

combined = nii_img_orig_t1ce[:, :, 65]

# k = 65

# for i in range(240):
#     for j in range(240):
# #         pass
# #         combined[i][j] = 0
#         if nii_img_tc_seg[i][j][k] != 0 and nii_img_et_seg[i][j][k] != 0:
#             combined[i][j] = 100
# #             print(1, i, j)
#             continue
#         if nii_img_tc_seg[i][j][k] != 0 and nii_img_et_seg[i][j][k] == 0:
#             combined[i][j] = 100
# #             print(2, i, j)
#             continue
#         if nii_img_tc_seg[i][j][k] == 0 and nii_img_et_seg[i][j][k] != 0:
#             combined[i][j] = 100
# #             print(3, i, j)
#             continue

plt.imshow(combined)
# plt.imshow(nii_img_tc_seg[:, :, 65], cmap='jet', alpha=0.25)
# plt.imshow(nii_img_et_seg[:, :, 65], cmap='jet', alpha=0.25)
plt.imshow(combined_mask)


In [ ]:
np.unique(combined_mask)